In [2]:
#Let's start by loading in our modules, setting up our connection to the database, and then testing that connection. 
#Importing
import os

import pandas as pd
from sqlalchemy import create_engine

from dotenv import load_dotenv
load_dotenv()

#Setting up Connection
username = os.getenv('POSTGRES_USERNAME')
password = os.getenv('POSTGRES_PASSWORD')


postgres_connection_string = "postgres://{username}:{password}@{host}:{port}/{database}?gssencmode=disable".format(
    username=username,
    password=password,
    host="localhost",
    port="5432",
    database="LahmanBaseball"
)

engine = create_engine(postgres_connection_string)

#Testing Connection
batting_sql = "SELECT * FROM batting;"

batting_df = pd.read_sql(batting_sql, con=engine)
batting_df.head()

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,h2b,...,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
0,abercda01,1871,1,TRO,NA,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,NaN


In [10]:
#Let's take a look at what tables we have
info_sql = "SELECT * FROM information_schema.tables"
info_df = pd.read_sql(info_sql, con=engine)
info_df = info_df.head(27)
info_df = info_df[['table_name']]
info_df

,table_name
0,allstarfull
1,appearances
2,awardsmanagers
3,awardsplayers
4,awardssharemanagers
5,awardsshareplayers
6,batting
7,battingpost
8,collegeplaying
9,fielding


What range of years does the provided database cover?

In [14]:
#We can continue to use our batting dataframe for this one
batting_df.yearid.min()

1871

Find the name and height of the shortest player in the database. How many games did he play in? What is the name of the team for which he played?

In [15]:
#Let's fetch the people table and look at the columns
people_sql = "SELECT * FROM people;"

people_df = pd.read_sql(people_sql, con=engine)
people_df.head()

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,...,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [21]:
#Now we can filter that down to only the people table info on the shortest player
shortest = people_df.height.min()
people_df[people_df['height'].isin([shortest])]

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,...,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
5843,gaedeed01,1925.0,6.0,8.0,USA,IL,Chicago,1961.0,6.0,18.0,...,Gaedel,Edward Carl,65.0,43.0,R,L,1951-08-19,1951-08-19,gaede101,gaedeed01


Find all players in the database who played at Vanderbilt University. Create a list showing each player’s first and last names as well as the total salary they earned in the major leagues. Sort this list in descending order by the total salary earned. Which Vanderbilt player earned the most money in the majors?


Find the average number of strikeouts per game by decade since 1920. Round the numbers you report to 2 decimal places. Do the same for home runs per game. Do you see any trends?

Find the player who had the most success stealing bases in 2016, where success is measured as the percentage of stolen base attempts which are successful. (A stolen base attempt results either in a stolen base or being caught stealing.) Consider only players who attempted at least 20 stolen bases.